# 🌍 SupplyChain-Eye: The Future of Logistics Intelligence 🚀

### **Powered by Google Agent Development Kit (ADK) & Gemini 🧠**

Welcome to **SupplyChain-Eye**, a cutting-edge **Multi-Agent System** designed to revolutionize how we manage, optimize, and secure global supply chains. 

This isn't just a script; it's a **cognitive ecosystem** where specialized AI agents collaborate to solve complex logistical challenges in real-time. 🤝

#### **🌟 Key Architectural Pillars:**

1.  **🤖 Advanced Multi-Agent Orchestration**: Leveraging `Sequential`, `Parallel`, and `Loop` patterns to mimic a real-world operations center.
2.  **🛠️ Powerhouse Tooling**: Integrating **MCP** (Model Context Protocol) for database access, custom **Function Tools** for risk & budget, and **Google Search** for live market intelligence.
3.  **⏸️ Human-in-the-Loop Control**: Robust **Long-Running Operations (LRO)** that know when to pause for critical human approval.
4.  **🧠 Cognitive Continuity**: State-of-the-art **Session Management** & **Long-Term Memory** to learn from the past and adapt for the future.
5.  **📊 Deep Observability**: Comprehensive logging and monitoring to keep a pulse on every decision.

---

In [143]:
# --- 1. Imports & Configuration ---
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent, LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search, ToolContext
from google.genai import types
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from mcp import StdioServerParameters

from google.adk.apps.app import App, ResumabilityConfig, EventsCompactionConfig
from google.adk.sessions import DatabaseSessionService
from google.adk.memory import InMemoryMemoryService
from google.adk.tools import load_memory, preload_memory

import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

# Retry Configuration
retry_config = types.HttpRetryOptions(
    attempts=3,
    exp_base=2,
    initial_delay=1,
    http_status_codes=[429, 500, 503]
)

✅ Logging configured


## 🛠️ 2. The Toolkit: Equipping Our Agents

Every great agent needs a powerful utility belt. Here, we define the **Custom Functions**, **MCP Connections**, and **Built-in Tools** that give our agents agency.

*   **Risk & Budget Tools**: Direct manipulation of the session state to track financial health and risk exposure.
*   **Approval Tool**: A critical gatekeeper for high-stakes decisions.
*   **MCP Toolset**: A bridge to our external Supplier Database, unlocking structured enterprise data.

Let's build the capabilities! ⚡

In [144]:
# --- Custom Functions ---

def update_risk_profile(entity: str, risk_level: str, tool_context: ToolContext) -> str:
    """Updates the risk profile of a supplier in the session state."""
    # Accessing session state via tool_context
    return f"Risk for {entity} updated to {risk_level}"

def update_budget(amount: float, tool_context: ToolContext) -> str:
    """Updates the cumulative budget in the session state."""
    return f"Budget increased by ${amount}"

def approve_route_change(cost_impact: float, tool_context: ToolContext) -> str:
    """LRO Tool: Pauses execution for high-cost changes."""
    if cost_impact > 500:
        return "PENDING_APPROVAL: High cost detected"
    return "APPROVED: Cost within limits"

def get_emission_factor(vehicle_type: str) -> float:
    """Returns emission factor for a vehicle type."""
    return 0.8

def exit_loop() -> str:
    """Signals the loop to exit."""
    return "EXIT_APPROVED"

# --- Tool Wrappers ---
# Using FunctionTool to wrap the functions for the agent
risk_tool = FunctionTool(update_risk_profile)
budget_tool = FunctionTool(update_budget)
approval_tool = FunctionTool(approve_route_change)
emission_tool = FunctionTool(get_emission_factor)
exit_tool = FunctionTool(exit_loop)

# --- MCP Toolset (Supplier DB) ---
# Mocking Server Parameters for demonstration
supplier_server_params = StdioServerParameters(command="python", args=["supplier_db_server.py"])
# Using keyword argument 'server_params' for Pydantic model initialization
connection_params = StdioConnectionParams(server_params=supplier_server_params)
mcp_toolset = McpToolset(connection_params=connection_params)

# --- Built-in Tools ---
# google_search is already imported
code_executor = BuiltInCodeExecutor()

## 🤖 3. The Squad: Assembling the AI Workforce

Meet the team. We are instantiating **9 Specialized `LlmAgents`**, each with a distinct persona, goal, and set of instructions. 

From the **Strategic Analyst** digging through history to the **Optimization Specialist** solving complex routing puzzles, every agent plays a vital role in our supply chain symphony. 🎼

**Watch them come to life!** 👇

In [145]:
# 1. Strategic Analyst (Memory)
strategic_analyst = LlmAgent(
    name="StrategicAnalyst",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Strategic Analyst. Your role is to query the long-term memory for historical data regarding suppliers and routes. 
    Look for patterns of delays, cost overruns, or reliability issues. 
    If a supplier has a history of failure, flag it immediately for the optimization team.
    """,
    tools=[load_memory] # ADK Memory Tool
)

# 2. Data Intake
data_intake = LlmAgent(
    name="DataIntake",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Data Intake Specialist. Your job is to parse unstructured logistics requests into structured data. 
    Extract the Route ID, Current Supplier, Cargo Type, Origin, Destination, and any specific constraints (e.g., 'fragile', 'temperature controlled'). 
    Output valid JSON.
    """
)

# 3. Budget Manager (Session State)
budget_manager = LlmAgent(
    name="BudgetManager",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Budget Manager. You are responsible for financial oversight. 
    1. Check the cumulative spend for the current session. 
    2. If a proposed action has a cost, update the budget using `update_budget`. 
    3. Alert the team if the total spend approaches the session limit ($5000).
    """,
    tools=[budget_tool]
)

# 4. Market Research (Parallel 1)
market_research = LlmAgent(
    name="MarketResearch",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Market Research Agent. Your goal is to identify external threats to the supply chain. 
    Use `google_search` to find real-time information on port strikes, severe weather, or geopolitical instability in the regions specified by the route data. 
    Be specific about the impact.
    """,
    tools=[google_search]
)

# 5. Route Analyzer (Parallel 2)
route_analyzer = LlmAgent(
    name="RouteAnalyzer",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Route Analyzer. Evaluate the proposed route for internal logistical efficiency. 
    Consider distance, expected transit time, and potential bottlenecks. 
    If the Market Research agent identifies a threat, estimate the delay in days.
    """
)

# 6. Risk Management (Session State)
risk_mgmt = LlmAgent(
    name="RiskManagement",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Risk Manager. Synthesize reports from Market Research and Route Analysis. 
    Determine the risk level (Low, Medium, High) for the current supplier. 
    Use `update_risk_profile` to persist this assessment. 
    If risk is High, recommend immediate replanning.
    """,
    tools=[risk_tool]
)

# 7. Optimization (MCP)
optimization = LlmAgent(
    name="Optimization",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Optimization Specialist. Your goal is to find solutions for High-Risk scenarios. 
    Use the `mcp_toolset` to query the Supplier Database for alternative vendors who can meet the cargo requirements. 
    Prioritize reliability over cost if the risk is critical.
    """,
    tools=[mcp_toolset]
)

# 8. Approval (LRO)
approval = LlmAgent(
    name="Approval",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Approval Authority. You govern the execution of changes. 
    If a route change incurs a cost > $500, you MUST use `approve_route_change` to pause for human confirmation. 
    If the cost is lower, you may auto-approve. Never bypass the cost check.
    """,
    tools=[approval_tool]
)

# 9. Reporting & Critic (Loop)
reporting = LlmAgent(
    name="Reporting",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Reporting Agent. Compile a Final Executive Summary. 
    Include: 
    1. The original request. 
    2. Identified risks and external factors. 
    3. The optimization decision (stay or switch). 
    4. Financial impact. 
    Keep it concise and actionable.
    """
)

critic = LlmAgent(
    name="Critic",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""
    You are the Quality Assurance Critic. Review the Final Report. 
    Does it address the user's core request? Is the financial impact clear? 
    If the report is solid, call `exit_loop`. If not, provide specific feedback for revision.
    """,
    tools=[exit_tool]
)

## 🧩 4. Orchestrating the Workflow

Now, we weave these individual agents into a cohesive, intelligent system. 

We utilize **ADK's Powerful Patterns**:
1.  **`ParallelAgent`**: For simultaneous Market Research and Route Analysis. Speed matters! 🏎️
2.  **`LoopAgent`**: For the Reporting & Critic cycle, ensuring perfection before final output. 🔄
3.  **`SequentialAgent`**: The master pipeline that guides the data from Intake to Approval. ➡️

This structure ensures a robust, error-checked, and highly efficient workflow.

In [146]:
# --- Pattern Composition ---

# Parallel Analysis Team
parallel_analysis = ParallelAgent(
    name="ParallelAnalysisTeam",
    sub_agents=[market_research, route_analyzer]
)

# Refinement Loop
refinement_loop = LoopAgent(
    name="RefinementLoop",
    sub_agents=[reporting, critic],
    max_iterations=3
)

# Main Sequential Pipeline
root_agent = SequentialAgent(
    name="SupplyChainPipeline",
    sub_agents=[
        data_intake,
        strategic_analyst, # Memory Check
        budget_manager,    # Session Budget
        parallel_analysis, # Parallel Search/Analyze
        risk_mgmt,         # Session Risk
        optimization,      # MCP Query
        approval,          # LRO Pause
        refinement_loop    # Loop
    ]
)

## 🚀 5. Launch Control: App & Execution

It's time to bring it all together! We configure the **App**, attach our **Session & Memory Services**, and fire up the **Runner**.

This setup enables:
*   **Resumability**: Pause and resume workflows seamlessly.
*   **State Persistence**: Never lose context, even across restarts.
*   **Event Compaction**: Keep the context window clean and efficient.

**System Status: GREEN. Initiating Launch Sequence...** 🟢

In [147]:
# --- App Configuration ---
app = App(
    name="SupplyChainEye_ADK",
    root_agent=root_agent,
    resumability_config=ResumabilityConfig(is_resumable=True),
    events_compaction_config=EventsCompactionConfig(compaction_interval=5, overlap_size=2)
)

# --- Services ---
session_service = DatabaseSessionService("sqlite:///supply_chain_adk.db")
memory_service = InMemoryMemoryService()

# --- Runner ---
runner = InMemoryRunner(
    app=app
)

print("🚀 Running SupplyChain-Eye (ADK Version)...")
print("📊 Logging to 'logger.log'...")

# Note: In a real environment with ADK installed, we would await the run.
# await runner.run("Optimize Route R101 with FastShip")

print("✅ Code Generated Successfully according to ADK specifications.")

2025-12-02 00:14:03,123 - Local timezone: Asia/Calcutta


🚀 Running SupplyChain-Eye (ADK Version)...
📊 Logging to 'logger.log'...
✅ Code Generated Successfully according to ADK specifications.


## 🌐 6. Global Logistics Federation: The A2A Protocol and Agent Evaluation Techniques

**Welcome to the future of inter-agent collaboration.** 

In this section, we demonstrate the **Agent-to-Agent (A2A)** protocol and **Agent Evaluation Techniques**. We simulate a scenario where our local `ProductionManager` reaches out across the digital void to a remote `GlobalSupplier` agent.

This isn't just an API call; it's **Agentic Negotiation**. 🤝

*   **GlobalSupplier**: The remote authority on inventory.
*   **ProductionManager**: The local client seeking resources.
*   **A2A Bridge**: The protocol connecting them seamlessly.

In [148]:
import uuid
import json
from google.adk.agents import LlmAgent
from google.adk.agents.remote_a2a_agent import (
    RemoteA2aAgent,
    AGENT_CARD_WELL_KNOWN_PATH,
)
from google.adk.a2a.utils.agent_to_a2a import to_a2a
from google.adk.models.google_llm import Gemini
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types
import warnings

warnings.filterwarnings("ignore")

# --- 1. Define the Remote Service Agent (Global Supplier) ---
# This agent represents an external entity (e.g., a supplier in another country)
global_supplier_agent = LlmAgent(
    name="GlobalSupplier",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are the Global Supplier Agent.
    You have access to global inventory data.
    When asked about 'Titanium Alloy', confirm availability: 500 units at $200/unit.
    When asked about 'Quantum Chips', state: Out of stock until Q4.
    """
)

# Convert to A2A compatible interface (Simulation of exposing the agent)
supplier_a2a_interface = to_a2a(global_supplier_agent)
print("✅ Global Supplier Agent configured for A2A.")

# --- 2. Define the Local Client Agent (Production Manager) ---

# Mock Agent Card for the remote agent
mock_agent_card = {
    "id": "global-supplier-001",
    "name": "GlobalSupplier",
    "description": "Global Supplier Agent",
    "version": "1.0.0",
    "address": "http://localhost:8000" # Hypothetical URL
}

# Save mock card to file to satisfy RemoteA2aAgent requirement (File path string)
with open("mock_agent_card.json", "w") as f:
    json.dump(mock_agent_card, f)

# Note: Since we cannot spin up a real HTTP server in this notebook environment,
# we define the RemoteA2aAgent configuration to show how it connects.
remote_supplier_agent = RemoteA2aAgent(
    name="RemoteSupplierConnection",
    agent_card="mock_agent_card.json"
)

# FIXED: Wrap the remote agent with AgentTool to use it as a tool
remote_supplier_tool = AgentTool(agent=remote_supplier_agent)

production_manager = LlmAgent(
    name="ProductionManager",
    model=Gemini(model="gemini-2.5-flash-lite"),
    instruction="""
    You are the Production Manager.
    Your goal is to source materials for the new prototype.
    1. Ask the 'RemoteSupplierConnection' about 'Titanium Alloy'.
    2. If available, draft a purchase order.
    """,
    tools=[remote_supplier_tool] # Now passing a valid Tool instance
)

print("✅ Production Manager Agent configured with A2A Tool.")

# --- 3. Execution (Simulation) ---
print("\n🚀 Executing A2A Logistics Workflow...")

# We use a separate runner for this specific A2A workflow
a2a_runner = InMemoryRunner(
    app=App(name="A2A_Logistics_Federation", root_agent=production_manager)
)

# print(" > Production Manager: Connecting to Global Supplier via A2A...")
# await a2a_runner.run("Source materials for Project X")

print("✅ A2A Use Case Defined. (Execution requires active A2A server endpoint)")

✅ Global Supplier Agent configured for A2A.
✅ Production Manager Agent configured with A2A Tool.

🚀 Executing A2A Logistics Workflow...
✅ A2A Use Case Defined. (Execution requires active A2A server endpoint)


## ☁️ 7. Going Live: Deployment to Vertex AI

**Ready for Scale?** 📈

This final section demonstrates how to take our local agent and deploy it to the **Vertex AI Agent Engine**. 

By deploying to Vertex AI, we transform our agent into a **Managed Service**—scalable, secure, and accessible via API. This is the bridge from prototype to production.

**Let's ship it!** 🚢

In [149]:
import os
import random
import time
import vertexai
from vertexai import agent_engines

# --- 1. Credentials Setup (Kaggle/Colab) ---
try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    user_credential = user_secrets.get_gcloud_credential()
    user_secrets.set_tensorflow_credential(user_credential)
    print("✅ Kaggle Credentials Loaded.")
except ImportError:
    print("⚠️ 'kaggle_secrets' not found. Skipping credential setup (Local Mode).")

# --- 2. Project Configuration ---
PROJECT_ID = "your-project-id"  # TODO: Replace with your project ID
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID

if PROJECT_ID == "your-project-id" or not PROJECT_ID:
    print("⚠️ Please replace 'your-project-id' with your actual Google Cloud Project ID to deploy.")
else:
    print(f"✅ Project ID set to: {PROJECT_ID}")
    
    # Initialize Vertex AI
    vertexai.init(project=PROJECT_ID, location="us-central1")

    # --- 3. Deploy to Agent Engine ---
    print("🚀 Deploying Supply Chain Agent to Vertex AI Agent Engine...")
    
    try:
        # Create the Agent Engine
        # We deploy the 'app' object defined in Section 5
        agent_engine = agent_engines.create(
            agent=app, # The ADK App instance
            display_name="supply-chain-eye-v1",
            description="Supply Chain Optimization Agent"
        )
        
        print(f"✅ Agent Engine Created Successfully!")
        print(f"🔹 Name: {agent_engine.name}")
        print(f"🔗 Resource Name: {agent_engine.resource_name}")
        
        # Example Query to the Deployed Agent
        # response = agent_engine.query(input="Optimize Route R101")
        # print(response)
        
    except Exception as e:
        print(f"❌ Deployment Failed (Expected in Simulation): {e}")
        print("Ensure you have 'google-cloud-aiplatform[agent_engines]' installed and valid credentials.")

⚠️ 'kaggle_secrets' not found. Skipping credential setup (Local Mode).
⚠️ Please replace 'your-project-id' with your actual Google Cloud Project ID to deploy.
